#### Econ 627: Assignment 4

##### <u> Question 1 </u>: 2SLS Bias due to Many IVs

In this question, you will use MC simulations to investigate the bias of the 2SLS estimator due to many IVs

<u> Part A </u>

###### Let's load the necessary libraries 

In [1]:
using Distributions, PrettyTables, Random, Parameters,DataFrames


In [3]:
function DGP(n,k₂)

    # The regression coefficients
    λ  = 0.5
    β₁ = 1
    β₂ = ones(k₂,1)
    π₁ = 1
    π₂ = ones(k₂,1)

   
    ϵ = randn(n,1)       # This produces a n x 1 matrix of numbers independentlly  drawn from a normal distribution
    V = randn(n,1)
    
    Z₁ = randn(n,1)      # Zᵢ  and Xᵢ₂ are independent standard normals
    X₂ = randn(n,k₂)
    
    U = λ*V + ϵ
    X₁ = π₁*Z₁ + X₂*π₂ + V
    Y = β₁*X₁ + X₂*β₂ + U

    return Y, X₁, X₂, Z₁
    
end


DGP (generic function with 1 method)

<u> Part B :</u> 
Simulation of the bias of the 2SLS estimator of β1 that uses $Z_{i,2}$ and  $X_{i,2}$ in the first stage.

###### First, let us define a function that compute the 2SLS estimators


In [55]:
function β₂ₛₗₛ(Y,X,Z)
    
    Q = X'*Z;
    B = inv(Z'*Z);
    
    return inv(Q*B*Q')*Q*B*(Z'*Y)
end

β₂ₛₗₛ (generic function with 1 method)

###### Simulated bias of the 2SLS estimator with few IVs,  𝑛=30
 

In [6]:
β = zeros(10^5,1)
Random.seed!(627);

for r = 1:10^5
    Y, X₁, X₂, Z₁ = DGP(30,2)
    X = [X₁ X₂]
    Z = [Z₁ X₂]
    β[r] = β₂ₛₗₛ(Y,X,Z)[1]
end
biasᵢᵥ = mean(β)-1 

-0.024317057362530203

<u> Part C :</u> 
Simulation with many IVs.

In [7]:
β = zeros(10^5,1)

Random.seed!(627);

for r=1:10^5
    Y, X₁, X₂, Z₁ = DGP(30,2)
    X=[X₁ X₂]
     
    Z=[Z₁ Z₁.^2 Z₁.^3 X₂ X₂.^2 X₂.^3 ([Z₁ Z₁] .* X₂) ([Z₁ Z₁].^2 .* X₂) ([Z₁ Z₁] .* X₂.^2) Z₁.*X₂[:,1].*X₂[:,2]]
    
    β[r]= β₂ₛₗₛ(Y,X,Z)[1]
end
biasₘᵢᵥₛ = mean(β) - 1


0.1611287131108996

<u> Part D :</u> 

###### We can see that the bias of the multiple IVs estimator is far greater than than that of IV estimator. This shows that the multiple IVs deteriorate the prediction power of the model.


<u> Part E:</u> 

Repeat Parts B-C with n=100,1000 and compare to the results in Parts B-C. Explain the differences.


In [8]:
result = []
i = 1
for n in [100, 1000]
    β = zeros(10^5,1)

    for r = 1:10^5
        Y, X₁, X₂, Z₁ = DGP(n,2)
        X = [X₁ X₂]
        Z = [Z₁ X₂]
        β[r] = β₂ₛₗₛ(Y,X,Z)[1]
    end
    bias = mean(β)-1 
    push!(result, bias)

    for r=1:10^5
        Y, X₁, X₂, Z₁ = DGP(n,2)
        X=[X₁ X₂]
        
        Z=[Z₁ Z₁.^2 Z₁.^3 X₂ X₂.^2 X₂.^3 ([Z₁ Z₁] .* X₂) ([Z₁ Z₁].^2 .* X₂) ([Z₁ Z₁] .* X₂.^2) Z₁.*X₂[:,1].*X₂[:,2]]
        
        β[r]= β₂ₛₗₛ(Y,X,Z)[1]
    end
    bias = mean(β)-1 
    push!(result, bias)
end


In [10]:
result

4-element Vector{Any}:
 -0.005427747662165472
  0.056043979939559785
 -0.0005786530844793525
  0.0059551805196893515

In [13]:
table_data = ["Bias for multiple IVs" result[1] result[3];
        "Bias for IVs" result[2] result[4];
]
header=[" "," n = 100","n = 1000"]
pretty_table(table_data;header)

┌───────────────────────┬─────────────┬──────────────┐
│                       │     n = 100 │     n = 1000 │
├───────────────────────┼─────────────┼──────────────┤
│ Bias for multiple IVs │ -0.00542775 │ -0.000578653 │
│          Bias for IVs │    0.056044 │   0.00595518 │
└───────────────────────┴─────────────┴──────────────┘


###### We observe that the detriment effect of many IVs on the estimation of β₂ₛₗₛ gradually wear off as the number of IVs is relatively smaller than the sample size. So, the model behaves as if we had fewer IVs.

#### <u> Question 2 </u>: Non-centrality parameter calculations
In this question, you will use numerical methods to compute the $\chi^2_1$ non-centrality parameter corresponding to 80% power.

<u> Part A :</u> 

Define a custom Julia function that for given c>0 and α∈(0,1) numerically computes the probability of rejection for 
the non-central $\chi^2_1$ distribution with the non-centrality parameter c>0.

In [15]:
using Distributions, QuadGK

In [36]:
function power1(c,α)

    power = quadgk(z-> ((c +z)^2 > quantile(Chisq(1),1-α)) * pdf(Normal(0,1),z),-Inf,Inf,atol=1e-8)[1]
    return power
end


power1 (generic function with 1 method)

In [37]:
power1(0.45, 0.05)

0.0735033553390831

<u> Part B:</u> 

In [41]:
function power2(c,α)

    dist = NoncentralChisq(1, c^2)
    power = 1 - cdf(dist, quantile(Chisq(1),1-α))
    return power
end

power2 (generic function with 1 method)

<u> Part C:</u> 

The non-centrality parameter corresponding to the power π = 0.8 and α =0.05.



In [54]:
using Roots

find_zero(c->power2(c,0.05)-0.8, (0,3), atol=1e-10)

2.801581786928864

#### <u> Question 3 </u>: Minimum Detectable Effects (MDEs)

<u> Part A: </u> 

search: NoncentralChisq NoncentralHypergeometric FisherNoncentralHypergeometric



```
NoncentralChisq(ν, λ)
```

The *noncentral chi-squared distribution* with `ν` degrees of freedom and noncentrality parameter `λ` has the probability density function

$$
f(x; \nu, \lambda) = \frac{1}{2} e^{-(x + \lambda)/2} \left( \frac{x}{\lambda} \right)^{\nu/4-1/2} I_{\nu/2-1}(\sqrt{\lambda x}), \quad x > 0
$$

It is the distribution of the sum of squares of `ν` independent [`Normal`](@ref) variates with individual means $\mu_i$ and

$$
\lambda = \sum_{i=1}^\nu \mu_i^2
$$

```julia
NoncentralChisq(ν, λ)     # Noncentral chi-squared distribution with ν degrees of freedom and noncentrality parameter λ

params(d)    # Get the parameters, i.e. (ν, λ)
```

External links

  * [Noncentral chi-squared distribution on Wikipedia](https://en.wikipedia.org/wiki/Noncentral_chi-squared_distribution)


In [22]:
sqrt(pi)

1.7724538509055159

search: cdf logcdf ccdf invlogcdf logccdf logdiffcdf invlogccdf Cptrdiff_t



```
cdf(d::UnivariateDistribution, x::Real)
```

Evaluate the cumulative probability at `x`.

See also [`ccdf`](@ref), [`logcdf`](@ref), and [`logccdf`](@ref).

---

```
cdf(d::Skellam, t::Real)
```

Implementation based on SciPy: https://github.com/scipy/scipy/blob/v0.15.1/scipy/stats/*discrete*distns.py

Refer to Eqn (5) in On an Extension of the Connexion Between Poisson and χ2 Distributions, N.L Johnson(1959) Vol 46, No 3/4, doi:10.2307/2333532 It relates the Skellam and Non-central chisquare PDFs, which is very similar to their CDFs computation as well.

Computing cdf of the Skellam distribution.

---

```
cdf(d, x)
```

Calculates the CDF of the distribution. To determine the CDF, the incomplete gamma function is required. The CDF  of the Gamma distribution provides this, with the necessary 1/Γ(a) normalization.
